In [1]:
import tensorflow as tf
from tensorflow import keras

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [2]:
device = "cuda" if tf.test.is_gpu_available() else "CPU"
print(device)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
cuda


In [25]:
# Benchmark Dataset
# Square1, Square2, Circle1, Circle2, Triangle1, Triangle2
NUM_CLASSES = 6

# Fixed for Cats & Dogs color images
CHANNELS = 3

IMAGE_RESIZE = 224
RESNET50_POOLING_AVERAGE = 'avg'
DENSE_LAYER_ACTIVATION = 'softmax'
OBJECTIVE_FUNCTION = 'categorical_crossentropy'

# Common accuracy metric for all outputs, but can use different metrics for different output
LOSS_METRICS = ['accuracy']

# EARLY_STOP_PATIENCE must be < NUM_EPOCHS
NUM_EPOCHS = 10
EARLY_STOP_PATIENCE = 3

# These steps value should be proper FACTOR of no.-of-images in train & valid folders respectively
# Training images processed in each step would be no.-of-train-images / STEPS_PER_EPOCH_TRAINING
STEPS_PER_EPOCH_TRAINING = 7
STEPS_PER_EPOCH_VALIDATION = 7

# These steps value should be proper FACTOR of no.-of-images in train & valid folders respectively
# NOTE that these BATCH* are for Keras ImageDataGenerator batching to fill epoch step input
BATCH_SIZE_TRAINING = 3
BATCH_SIZE_VALIDATION = 3

# Using 1 to easily manage mapping between test_generator & prediction for submission preparation
BATCH_SIZE_TESTING = 1

In [26]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import tensorflow as tf
print(tf.__version__)

2.3.0


In [27]:
# Still not talking about our train/test data or any pre-processing.
model = Sequential()

# 1st layer as the lumpsum weights from resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
# NOTE that this layer will be set below as NOT TRAINABLE, i.e., use it as is
model.add(ResNet50(include_top = False, pooling = RESNET50_POOLING_AVERAGE, weights = 'imagenet'))

# 2nd layer as Dense for 2-class classification, i.e., dog or cat using SoftMax activation
model.add(Dense(NUM_CLASSES, activation = DENSE_LAYER_ACTIVATION))

# Say not to train first layer (ResNet) model as it is already trained
# model.layers[0].trainable = False

In [28]:
from tensorflow.keras import optimizers

sgd = optimizers.SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(optimizer = sgd, loss = OBJECTIVE_FUNCTION, metrics = LOSS_METRICS)

In [29]:
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_size = IMAGE_RESIZE

# preprocessing_function is applied on each image but only after re-sizing & augmentation (resize => augment => pre-process)
# Each of the keras.application.resnet* preprocess_input MOSTLY mean BATCH NORMALIZATION (applied on each batch) stabilize the inputs to nonlinear activation functions
# Batch Normalization helps in faster convergence
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   horizontal_flip=True,
                                   vertical_flip=True)

# flow_From_directory generates batches of augmented data (where augmentation can be color conversion, etc)
# Both train & valid folders must have NUM_CLASSES sub-folders
train_generator = data_generator.flow_from_directory(
        './data/Benchmarkimage_20200821/train',
        target_size=(image_size, image_size),
        batch_size=BATCH_SIZE_TRAINING,
        class_mode='categorical')

validation_generator = data_generator.flow_from_directory(
        './data/Benchmarkimage_20200821/val/',
        target_size=(image_size, image_size),
        batch_size=BATCH_SIZE_VALIDATION,
        class_mode='categorical') 

Found 103 images belonging to 6 classes.
Found 18 images belonging to 6 classes.


In [30]:
# Early stopping & checkpointing the best model in ./working dir & restoring that as our model for prediction
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

cb_early_stopper = EarlyStopping(monitor = 'val_loss', patience = EARLY_STOP_PATIENCE)
cb_checkpointer = ModelCheckpoint(filepath = './working/best.hdf5', monitor = 'val_loss', save_best_only = True, mode = 'auto')


In [31]:
fit_history = model.fit_generator(
        train_generator,
        steps_per_epoch=STEPS_PER_EPOCH_TRAINING,
        epochs = NUM_EPOCHS,
        validation_data=validation_generator,
        validation_steps=STEPS_PER_EPOCH_VALIDATION,
#        callbacks=[cb_checkpointer, cb_early_stopper]
)
# model.load_weights("./working/best.hdf5")

Epoch 1/10
7/7 [==============================] - 7s 976ms/step - loss: 2.6433 - accuracy: 0.3810 - val_loss: 85.5173 - val_accuracy: 0.0000e+00
Epoch 2/10
7/7 [==============================] - 3s 369ms/step - loss: 3.1137 - accuracy: 0.2857
Epoch 3/10
7/7 [==============================] - 3s 473ms/step - loss: 2.2847 - accuracy: 0.3810
Epoch 4/10
7/7 [==============================] - 4s 506ms/step - loss: 1.9756 - accuracy: 0.4286
Epoch 5/10
7/7 [==============================] - 2s 337ms/step - loss: 1.7677 - accuracy: 0.3810
Epoch 6/10
7/7 [==============================] - 3s 369ms/step - loss: 1.6364 - accuracy: 0.5789
Epoch 7/10
7/7 [==============================] - 2s 275ms/step - loss: 1.7308 - accuracy: 0.3684
Epoch 8/10
7/7 [==============================] - 3s 404ms/step - loss: 2.7075 - accuracy: 0.1579
Epoch 9/10
7/7 [==============================] - 3s 361ms/step - loss: 1.9520 - accuracy: 0.2105
Epoch 10/10
7/7 [==============================] - 3s 403ms/step - loss

In [32]:
fit_history.history

{'loss': [2.643294095993042,
  3.1136507987976074,
  2.2846925258636475,
  1.9755979776382446,
  1.7676609754562378,
  1.636400818824768,
  1.7307558059692383,
  2.707475185394287,
  1.9520468711853027,
  1.7206684350967407],
 'accuracy': [0.380952388048172,
  0.2857142984867096,
  0.380952388048172,
  0.4285714328289032,
  0.380952388048172,
  0.5789473652839661,
  0.3684210479259491,
  0.15789473056793213,
  0.21052631735801697,
  0.3684210479259491],
 'val_loss': [85.51728057861328],
 'val_accuracy': [0.0]}

In [19]:
test_generator = data_generator.flow_from_directory(
        './data/Benchmarkimage_20200821/test/',
        target_size=(image_size, image_size),
        batch_size=BATCH_SIZE_TESTING,
        class_mode='categorical') 

Found 18 images belonging to 6 classes.


In [20]:
scores = model.evaluate(test_generator,
                        steps = 5)

5/5 [==============================] - 1s 105ms/step - loss: 18012970.0000 - accuracy: 0.0000e+00
